In [1]:
!pip install gdown

# Unduh file dari Google Drive
!gdown --id 1iePr-95ohkXLcgaXBEWNYb2R3lu_-pjz -O daily-min-temperatures.csv


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1iePr-95ohkXLcgaXBEWNYb2R3lu_-pjz
To: /content/daily-min-temperatures.csv
100% 67.9k/67.9k [00:00<00:00, 4.53MB/s]


In [3]:
import pandas as pd
data = pd.read_csv("daily-min-temperatures.csv")
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data['Temp_scaled'] = scaler.fit_transform(data[['Temp']])

In [5]:
data['Lag1'] = data['Temp_scaled'].shift(1)
data['Lag2'] = data['Temp_scaled'].shift(2)
data.dropna(inplace=True)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [7]:
X = data[['Lag1', 'Lag2']].values
y = data['Temp_scaled'].values

In [8]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [9]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [10]:
# Bangun model LSTM
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    Dense(1)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(optimizer='adam', loss='mse')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.1408 - val_loss: 0.0136
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0162 - val_loss: 0.0113
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0137 - val_loss: 0.0095
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0123 - val_loss: 0.0087
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0124 - val_loss: 0.0087
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0124 - val_loss: 0.0085
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0124 - val_loss: 0.0087
Epoch 8/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0122 - val_loss: 0.0087
Epoch 9/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0119 - val_loss: 0.0084
Epoch 10/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0115 - val_loss: 0.0084
Epoch 11/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0117 - val_loss: 0.0085
Epoch 12/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0118 - val_

In [13]:
import tensorflow as tf
import numpy as np

# Misalnya, y_true dan y_pred adalah nilai aktual dan prediksi
def r2_score_tf(y_true, y_pred):
    y_true_mean = tf.reduce_mean(y_true)
    ss_total = tf.reduce_sum(tf.square(y_true - y_true_mean))
    ss_residual = tf.reduce_sum(tf.square(y_true - y_pred))
    r2 = 1 - (ss_residual / ss_total)
    return r2

def rmse_tf(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [14]:
# Evaluasi
y_pred = model.predict(X_test).flatten()

# Hitung metrik
y_test = tf.constant(y_test, dtype=tf.float32)
y_pred = tf.constant(y_pred, dtype=tf.float32)

r2 = r2_score_tf(y_test, y_pred)
rmse = rmse_tf(y_test, y_pred)

print(f"R² Score: {r2.numpy():.4f}")
print(f"RMSE: {rmse.numpy():.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
R² Score: 0.6624
RMSE: 0.0907
